In [10]:
import os
import optuna
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [11]:
# Step 1: Load FLIR images into an array
def load_images_from_folder(folder_path, img_size=(64, 64)):
    images = []
    labels = []
    class_names = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    for label in class_names:
        class_folder = os.path.join(folder_path, label)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, img_size)
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels), class_names

# Replace 'dataset_path' with your dataset folder path
dataset_path = '/Python/magang/Thermal_Image_Analysis/sawit/'
images, labels, class_names = load_images_from_folder(dataset_path)

In [12]:
# Check if class_names is not empty before using it
if class_names:
    # Step 2: Labeling the images based on the folder
    class_map = {class_name: idx for idx, class_name in enumerate(class_names)}
    labels = np.array([class_map[label] for label in labels])

    # Normalize the images
    images = images / 255.0

    # One-hot encode labels
    labels = tf.keras.utils.to_categorical(labels, num_classes=len(class_names))
else:
    print("No classes found in the dataset folder.")
# Step 3: Define the objective function for Optuna
def objective(trial):
    # Define hyperparameters to optimize
    num_filters = trial.suggest_int('num_filters', 16, 64)
    kernel_size = trial.suggest_int('kernel_size', 3, 5)
    dense_units = trial.suggest_int('dense_units', 64, 256)

    # Build the model
    model = Sequential([
        Conv2D(num_filters, (kernel_size, kernel_size), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(dense_units, activation='relu'),
        Dense(len(class_names), activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

    # Create image data generators
    datagen = ImageDataGenerator()

    # Train the model
    model.fit(datagen.flow(X_train, y_train), validation_data=(X_val, y_val), epochs=10, verbose=0)

    # Evaluate the model on validation data
    accuracy = model.evaluate(X_val, y_val, verbose=0)[1]
    
    return accuracy

# Step 4: Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Print the best hyperparameters and their corresponding accuracy
best_params = study.best_params
best_accuracy = study.best_value
print("Best hyperparameters:", best_params)
print("Best accuracy:", best_accuracy)

# Step 5: Train the final model with the best hyperparameters
best_num_filters = best_params['num_filters']
best_kernel_size = best_params['kernel_size']
best_dense_units = best_params['dense_units']

final_model = Sequential([
    Conv2D(best_num_filters, (best_kernel_size, best_kernel_size), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(best_dense_units, activation='relu'),
    Dense(len(class_names), activation='softmax')
])

final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

final_model.fit(images, labels, epochs=10)  # Train the model on the entire dataset

# Step 6: Test the model with a single image
def test_single_image(model, image_path, img_size=(64, 64)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, img_size)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    prediction = model.predict(img)
    predicted_class = class_names[np.argmax(prediction)]
    return predicted_class

[I 2024-05-27 19:59:29,997] A new study created in memory with name: no-name-a8899802-d642-4f94-8c45-2bef5377f51c
c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
[I 2024-05-27 20:02:46,386] Trial 0 finished with value: 0.533544659614563 and parameters: {'num_filter

Best hyperparameters: {'num_filters': 47, 'kernel_size': 3, 'dense_units': 229}
Best accuracy: 0.6117274165153503
Epoch 1/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 56s 184ms/step - accuracy: 0.1955 - loss: 4.2117
Epoch 2/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 55s 185ms/step - accuracy: 0.2851 - loss: 2.8459
Epoch 3/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 55s 184ms/step - accuracy: 0.3890 - loss: 2.2747
Epoch 4/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 54s 183ms/step - accuracy: 0.4471 - loss: 2.0148
Epoch 5/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 55s 187ms/step - accuracy: 0.4890 - loss: 1.7826
Epoch 6/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 55s 186ms/step - accuracy: 0.5352 - loss: 1.6332
Epoch 7/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 55s 187ms/step - accuracy: 0.5893 - loss: 1.4567
Epoch 8/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 58s 195ms/step - accuracy: 0.6205 - loss: 1.3146
Epoch 9/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 59s 198ms/step - accuracy: 0.6457 - loss: 1.2114
Epoch 10/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 56s 190ms/step - accuracy: 0.6917 - loss: 1.

In [14]:
# Replace 'test_image_path' with the path to your test image
test_image_path = '/Python/magang/Thermal_Image_Analysis/sawit/ab 1/FLIR2603.jpg'
predicted_class = test_single_image(final_model, test_image_path)

print(f'The predicted class for the test image is: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
The predicted class for the test image is: ab 1
